#### import Splinter and BeautifulSoup

In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd

#### set executable path

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [C:\Users\PNCSI\.wdm\drivers\chromedriver\win32\94.0.4606.61\chromedriver.exe] found in cache


#### assign the url and instruct the browser to visit it

In [3]:
## 1. searching for elements with a specific combination of tag (div) and attribute (list_text). As an example, ul.item_list 
##    would be found in HTML as <ul class="item_list">.
## 2. Secondly, we're also telling our browser to wait one second before searching for components. The optional delay is 
##    useful because sometimes dynamic pages take a little while to load, especially if they are image-heavy.

In [4]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

#### set up the HTML parser:

In [5]:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [6]:
slide_elem

<div class="list_text">
<div class="list_date">October 16, 2021</div>
<div class="content_title">NASA's InSight Flexes Its Arm While Its 'Mole' Hits Pause</div>
<div class="article_teaser_body">Now that the lander's robotic arm has helped the mole get underground, it will resume science activities that have been on hold.</div>
</div>

In [7]:
slide_elem.find('div', class_='content_title')

<div class="content_title">NASA's InSight Flexes Its Arm While Its 'Mole' Hits Pause</div>

In [8]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"NASA's InSight Flexes Its Arm While Its 'Mole' Hits Pause"

In [9]:
# Get article summary
art_summ = slide_elem.find('div', class_='article_teaser_body').get_text()
art_summ

"Now that the lander's robotic arm has helped the mole get underground, it will resume science activities that have been on hold."

### Featured Images

In [10]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [11]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [12]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [13]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

In [14]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

### Mars Facts

In [15]:
# Get the infor from a table 
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [16]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [17]:
##browser.quit()

## D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [18]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'
browser.visit(url)

In [19]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
img_soup = soup(browser.html, 'html.parser')
titles = [i.text for i in img_soup.find_all('h3')[:-1]]
len(titles)

#urls={}
urls =[]
for i in titles:
    browser.find_by_text(i).click()
    url_soup = soup(browser.html, 'html.parser')
    url_complete = url + url_soup.find('a',text='Sample').get('href')
#    urls[i] = url_complete
    urls.append(url_complete)
    browser.back()

#hemisphere_image_urls = [{'Title': i, 'Url': j} for i,j in zip(titles, urls)]
hemisphere_image_urls = [{'img_url': j, 'title': i} for i,j in zip(titles, urls)]

In [20]:
import pandas as pd
pd.DataFrame(hemisphere_image_urls)

,img_url,title
0,https://marshemispheres.com/images/full.jpg,Cerberus Hemisphere Enhanced
1,https://marshemispheres.com/images/schiaparell...,Schiaparelli Hemisphere Enhanced
2,https://marshemispheres.com/images/syrtis_majo...,Syrtis Major Hemisphere Enhanced
3,https://marshemispheres.com/images/valles_mari...,Valles Marineris Hemisphere Enhanced


In [21]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'img_url': 'https://marshemispheres.com/images/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [22]:
# 5. Quit the browser
browser.quit()